# DCGAN - Deep Convolutional GANs

- [DCGAN paper link](https://arxiv.org/abs/1511.06434)

DCGAN (Deep Convolutional Generative Adversarial Network) has several unique features that distinguish it from traditional GAN architectures. Here are some of the key characteristics of DCGAN:

- Convolutional Architecture: DCGAN utilizes convolutional layers in both the generator and discriminator networks instead of fully connected layers. This allows the model to effectively capture spatial information and generate high-resolution images.

- Strided Convolutions and Transposed Convolutions: DCGAN uses strided convolutions in the discriminator to downsample the input and transposed convolutions in the generator to upsample the noise input. This helps in learning hierarchical representations and generating higher-resolution images.

- Batch Normalization: DCGAN applies batch normalization to the activations in both the generator and discriminator networks. It helps in stabilizing the training process and accelerates convergence by normalizing the inputs to each layer.

- LeakyReLU Activation: DCGAN employs LeakyReLU activation in the discriminator network instead of traditional ReLU. LeakyReLU allows for non-zero gradients for negative inputs, preventing the "dying ReLU" problem and improving the flow of gradients during training.

- No Fully Connected Layers: DCGAN does not use fully connected layers in the discriminator or generator. Instead, it relies on convolutional layers and global pooling to reduce the spatial dimensions of the activations.

- Random Noise Input: DCGAN takes random noise as input to the generator network. This noise vector is typically sampled from a uniform or normal distribution and serves as the source of randomness for generating diverse images.

These unique characteristics make DCGAN well-suited for generating high-quality and realistic images. It has been widely used in various applications, including image synthesis, image-to-image translation, and style transfer.








In [48]:
# import libraries

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [49]:
# Generator Model
def Generator():

    # input 
    input = tf.keras.Input(shape=(100,))   # accepts the shape of latent dimensions and batch size

    # Stacking layers
    dense_1 = tf.keras.layers.Dense(units=7*7*256, use_bias=False)(input)    # transform the input noise to higher dimensions representations
    reshape = tf.keras.layers.Reshape((7,7,256))(dense_1)      #  Reshapes the tensor to (7, 7, 256)

    conv2dT_1 = tf.keras.layers.Conv2DTranspose(filters=128, kernel_size=(5,5),
                                                 strides=(1,1),padding="same", use_bias=False)(reshape) # Performs transpose convolution on the input tensor, increasing its spatial dimensions.
    batchnorm_1 = tf.keras.layers.BatchNormalization()(conv2dT_1)
    leakyrelu_1 = tf.keras.layers.LeakyReLU(alpha=0.01)(batchnorm_1)


    conv2dT_2 = tf.keras.layers.Conv2DTranspose(filters=64, kernel_size=(5,5),
                                                 strides=(2,2),padding="same", use_bias=False)(leakyrelu_1) 
    batchnorm_2 = tf.keras.layers.BatchNormalization()(conv2dT_2)
    leakyrelu_2 = tf.keras.layers.LeakyReLU(alpha=0.01)(batchnorm_2)

    # Output layer with tanh activation
    generated_image = tf.keras.layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False,
                                             activation='tanh')(leakyrelu_2)
    
    # Defining Model
    model = tf.keras.Model(inputs=input, outputs=generated_image)
    
    return model


In [50]:
g = Generator()
g.summary()

Model: "model_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_23 (InputLayer)       [(None, 100)]             0         
                                                                 
 dense_20 (Dense)            (None, 12544)             1254400   
                                                                 
 reshape_11 (Reshape)        (None, 7, 7, 256)         0         
                                                                 
 conv2d_transpose_33 (Conv2D  (None, 7, 7, 128)        819200    
 Transpose)                                                      
                                                                 
 batch_normalization_46 (Bat  (None, 7, 7, 128)        512       
 chNormalization)                                                
                                                                 
 leaky_re_lu_50 (LeakyReLU)  (None, 7, 7, 128)         0  

In [51]:
# Discrminator 

def Discriminator():

    # input
    input = tf.keras.layers.Input(shape=(28,28,1))

    # stacking layers
    conv_1 = tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3))(input)
    batchnorm_1 = tf.keras.layers.BatchNormalization()(conv_1)
    leakyrelu_1 = tf.keras.layers.LeakyReLU(alpha=0.01)(batchnorm_1)
    droput_1 = tf.keras.layers.Dropout(rate=0.3)(leakyrelu_1)

    conv_2 = tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3))(droput_1)
    batchnorm_2 = tf.keras.layers.BatchNormalization()(conv_2)
    leakyrelu_2 = tf.keras.layers.LeakyReLU(alpha=0.01)(batchnorm_2)
    droput_2 = tf.keras.layers.Dropout(rate=0.3)(leakyrelu_2)

    conv_3 = tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3))(droput_2)
    batchnorm_3 = tf.keras.layers.BatchNormalization()(conv_3)
    leakyrelu_3 = tf.keras.layers.LeakyReLU(alpha=0.01)(batchnorm_3)
    droput_3 = tf.keras.layers.Dropout(rate=0.3)(leakyrelu_3)

    # Flatten
    flatten = tf.keras.layers.Flatten()(droput_3)

    output = tf.keras.layers.Dense(1, activation='sigmoid')(flatten)


    # Defining Model
    discriminator = tf.keras.models.Model(inputs=input, outputs=output)

    return discriminator


In [52]:
d = Discriminator()
d.summary()

Model: "model_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_24 (InputLayer)       [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_25 (Conv2D)          (None, 26, 26, 64)        640       
                                                                 
 batch_normalization_48 (Bat  (None, 26, 26, 64)       256       
 chNormalization)                                                
                                                                 
 leaky_re_lu_52 (LeakyReLU)  (None, 26, 26, 64)        0         
                                                                 
 dropout_25 (Dropout)        (None, 26, 26, 64)        0         
                                                                 
 conv2d_26 (Conv2D)          (None, 24, 24, 128)       73856     
                                                          

In [53]:
img = tf.random.normal((32,28,28,1))

d.predict(img).shape

1/1 [==============================] - 0s 105ms/step


(32, 1)

In [54]:
# Generate Noise

def generate_noise(latent_dimensions=100, batch_size=32):

    return tf.random.normal((batch_size,latent_dimensions))

In [62]:
# loss function
cross_entrophy = tf.keras.losses.BinaryCrossentropy()


# Generator Loss
def generator_loss(generator_output):

    return cross_entrophy(tf.ones_like(generator_output), generator_output)

# Discriminator loss
def discriminator_loss(fake_images, real_images):

    fake_loss = cross_entrophy(tf.zeros_like(fake_images), fake_images)
    real_loss = cross_entrophy(tf.ones_like(real_images),real_images )

    loss = fake_loss + real_loss

    return loss

- The generator's loss quantifies how well it was able to trick the discriminator. Intuitively, if the generator is performing well, the discriminator will classify the fake images as real (or 1). Here, compare the discriminators decisions on the generated images to an array of 1s.

- This method quantifies how well the discriminator is able to distinguish real images from fakes. It compares the discriminator's predictions on real images to an array of 1s, and the discriminator's predictions on fake (generated) images to an array of 0s.

In [63]:
# Optimizer

generator_optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

In [64]:
# Hyperparameters

epochs = 50
latent_dimensions = 100
batch_size = 64
BUFFER_SIZE = 60000

In [65]:
# data Loading & preprocessing

(train_images, train_labels), (_, _) = tf.keras.datasets.mnist.load_data()
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
train_images = (train_images - 127.5) / 127.5  # Normalize the images to [-1, 1]

train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(batch_size=batch_size)

In [66]:
# training step

generator = Generator()
discrminator = Discriminator()

@tf.function
def train_step(images):

    # generate noise
    noise = generate_noise(latent_dimensions=100, batch_size=batch_size)

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:

        # generator takes input as noise and generate image
        generated_images = generator(noise, training=True)

        # discrminator classify the fake and real images
        fake_clasiifier = discrminator(generated_images, training=True)
        real_classifier = discrminator(images, training=True)

        # Caluclate loss
        gen_loss = generator_loss(generated_images)
        disc_loss = discriminator_loss(fake_clasiifier, real_classifier)

    # Caluclate Gradients
    generator_gradients = gen_tape.gradient(gen_loss, generator.trainable_variables)
    discrminator_gradients = disc_tape.gradient(disc_loss, discrminator.trainable_variables)

    # optimizer step
    generator_optimizer.apply_gradients(zip(generator_gradients, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(discrminator_gradients, discrminator.trainable_variables))

In [69]:
# Train loop

def train(dataset, epochs=20):
    
    for epoch in range(epochs):
        
        for image_batch in dataset:
            train_step(image_batch)

        if epoch:

            print(f"Current epoch is {epoch}")

            # Generate samples after each epoch
            random_latent_vectors = tf.random.normal([10, 100])
            generated_images = generator(random_latent_vectors, training=False)

            # Reshape the generated images
            generated_images = generated_images.numpy().reshape(-1, 28, 28)

            # Display the generated images
            plt.figure(figsize=(10, 1))
            for i in range(10):
                plt.subplot(1, 10, i + 1)
                plt.imshow(generated_images[i], cmap='gray')
                plt.axis('off')
            plt.show()

In [ ]:
train(dataset=train_dataset)